In [1]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
df = pd.read_csv('compras.csv')
df = df.drop(columns=['cliente'])
df.stack().unique()

array(['limão', 'peito de peru', 'manga', 'cerveja', 'picanha', 'carvão',
       'sal', 'linguiça', 'mel', 'geleia', 'cereal', 'salsicha',
       'catchup', 'cebola', 'queijo', 'frutas', 'chá', 'alface',
       'granola', 'feijão', 'farinha de trigo', 'suco de laranja',
       'pimenta', 'sardinha enlatada', 'barra de cereal',
       'molho de tomate', 'gelatina', 'vinho branco', 'pão',
       'tempero pronto', 'vinho tinto', 'presunto', 'arroz', 'detergente',
       'filé de frango', 'papel higiênico', 'salada', 'cenoura',
       'maionese', 'molho branco', 'whisky', 'pudim', 'suco', 'azeitona',
       'frango', 'mostarda', 'carne moída', 'nhoque', 'água de coco',
       'laranja', 'cerveja artesanal', 'massa de pizza', 'biscoito',
       'morango', 'bisnaguinha', 'kiwi', 'achocolatado', 'sabão',
       'torrada', 'banana', 'palmito', 'desinfetante', 'sopa pronta',
       'ervilha', 'hambúrguer', 'batata frita', 'refrigerante',
       'queijo ralado', 'uva', 'óleo de soja', 'vodka', '

In [3]:
#limpando o dataset

df.replace(to_replace='vinho tinto', value='vinho', inplace=True)
df.replace(to_replace='vinho branco', value='vinho', inplace=True)
df.replace(to_replace='alface', value='verdura', inplace=True)

df.stack().unique()

array(['limão', 'peito de peru', 'manga', 'cerveja', 'picanha', 'carvão',
       'sal', 'linguiça', 'mel', 'geleia', 'cereal', 'salsicha',
       'catchup', 'cebola', 'queijo', 'frutas', 'chá', 'verdura',
       'granola', 'feijão', 'farinha de trigo', 'suco de laranja',
       'pimenta', 'sardinha enlatada', 'barra de cereal',
       'molho de tomate', 'gelatina', 'vinho', 'pão', 'tempero pronto',
       'presunto', 'arroz', 'detergente', 'filé de frango',
       'papel higiênico', 'salada', 'cenoura', 'maionese', 'molho branco',
       'whisky', 'pudim', 'suco', 'azeitona', 'frango', 'mostarda',
       'carne moída', 'nhoque', 'água de coco', 'laranja',
       'cerveja artesanal', 'massa de pizza', 'biscoito', 'morango',
       'bisnaguinha', 'kiwi', 'achocolatado', 'sabão', 'torrada',
       'banana', 'palmito', 'desinfetante', 'sopa pronta', 'ervilha',
       'hambúrguer', 'batata frita', 'refrigerante', 'queijo ralado',
       'uva', 'óleo de soja', 'vodka', 'batata', 'bolo', 'açú

In [4]:
itens_mais_comprados = df.stack().value_counts()
itens_mais_comprados.head(20)

queijo               1241
vinho                1178
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
salada                837
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
verdura               618
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
Name: count, dtype: int64

In [5]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

regras = apriori(transactions=transactions, min_support=0.008, min_confidence=0.5, min_lift=3, min_length=2, max_length=2)

resultado = list(regras)
resultado

[RelationRecord(items=frozenset({'arroz', 'feijão'}), support=0.0544, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'feijão'}), confidence=0.6384976525821596, lift=6.223174001775435), OrderedStatistic(items_base=frozenset({'feijão'}), items_add=frozenset({'arroz'}), confidence=0.530214424951267, lift=6.223174001775435)]),
 RelationRecord(items=frozenset({'filé de frango', 'arroz'}), support=0.0249, ordered_statistics=[OrderedStatistic(items_base=frozenset({'filé de frango'}), items_add=frozenset({'arroz'}), confidence=0.6587301587301587, lift=7.731574632983084)]),
 RelationRecord(items=frozenset({'frango', 'arroz'}), support=0.0425, ordered_statistics=[OrderedStatistic(items_base=frozenset({'frango'}), items_add=frozenset({'arroz'}), confidence=0.7508833922261485, lift=8.813185354767002)]),
 RelationRecord(items=frozenset({'café', 'açúcar'}), support=0.0238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'café'}), items_add=froze

In [6]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df_onehot, min_support=0.008,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)


In [7]:
rules = rules.sort_values(by='lift', ascending=False)
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
211,"(tomate, salada)","(cenoura, verdura)",0.0220,0.0129,0.0083,0.377273,29.245948,0.008016,1.585124,0.987533
206,"(cenoura, verdura)","(tomate, salada)",0.0129,0.0220,0.0083,0.643411,29.245948,0.008016,2.742652,0.978429
106,"(manteiga, suco de laranja)","(bolo, pão)",0.0196,0.0152,0.0087,0.443878,29.202470,0.008402,1.770833,0.985064
111,"(bolo, pão)","(manteiga, suco de laranja)",0.0152,0.0196,0.0087,0.572368,29.202470,0.008402,2.292628,0.980662
210,"(verdura, tomate)","(cenoura, salada)",0.0160,0.0179,0.0083,0.518750,28.980447,0.008014,2.040727,0.981193
207,"(cenoura, salada)","(verdura, tomate)",0.0179,0.0160,0.0083,0.463687,28.980447,0.008014,1.834750,0.983091
209,"(verdura, salada)","(cenoura, tomate)",0.0254,0.0126,0.0083,0.326772,25.934258,0.007980,1.466664,0.986498
208,"(cenoura, tomate)","(verdura, salada)",0.0126,0.0254,0.0083,0.658730,25.934258,0.007980,2.855805,0.973710
331,"(mostarda, batata frita)",(hambúrguer),0.0092,0.0348,0.0081,0.880435,25.299850,0.007780,8.072582,0.969392
334,(hambúrguer),"(mostarda, batata frita)",0.0348,0.0092,0.0081,0.232759,25.299850,0.007780,1.291380,0.995104


In [8]:
rules = rules.sort_values(by='lift', ascending=True)
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
47,"(presunto, queijo)",(geleia),0.0576,0.0483,0.0085,0.147569,3.055268,0.005718,1.116455,0.713812
50,(geleia),"(presunto, queijo)",0.0483,0.0576,0.0085,0.175983,3.055268,0.005718,1.143666,0.706837
296,(cerveja),(batata frita),0.0754,0.0600,0.0141,0.187003,3.116711,0.009576,1.156215,0.734533
297,(batata frita),(cerveja),0.0600,0.0754,0.0141,0.235000,3.116711,0.009576,1.208627,0.722499
371,(açúcar),(leite),0.0624,0.0707,0.0138,0.221154,3.128060,0.009388,1.193175,0.725590
370,(leite),(açúcar),0.0707,0.0624,0.0138,0.195191,3.128060,0.009388,1.164997,0.732070
343,(vodka),(cerveja),0.0501,0.0754,0.0120,0.239521,3.176671,0.008222,1.215813,0.721344
342,(cerveja),(vodka),0.0754,0.0501,0.0120,0.159151,3.176671,0.008222,1.129692,0.741083
294,(mostarda),(batata frita),0.0461,0.0600,0.0092,0.199566,3.326103,0.006434,1.174363,0.733146
295,(batata frita),(mostarda),0.0600,0.0461,0.0092,0.153333,3.326103,0.006434,1.126654,0.743987
